In [1]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

In [2]:
#define the dataframe columns
column_names = ['Postal_Code','Borough', 'Neighborhood'] 

Nebr = pd.DataFrame(columns=column_names)

# 1. Download and explore dataset

In [3]:
wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

page = urlopen(wiki)

from bs4 import BeautifulSoup
soup = BeautifulSoup(page,"lxml")
print(soup.prettify())

NameError: name 'urlopen' is not defined

In [4]:
Toronto=soup.find('table', class_='wikitable sortable')
Toronto

NameError: name 'soup' is not defined

In [5]:
#Generate lists
Pos=[]
Bor=[]
Neig=[]

for row in Toronto.findAll("tr"):
    cells = row.findAll('td')
    if len(cells)==3: 
        Pos.append(cells[0].find(text=True))
        Bor.append(cells[1].find(text=True))
        Neig.append(cells[2].find(text=True))

        
#Add Data to our DataFrame
Nebr['Postal_Code']=Pos
Nebr['Borough']=Bor
Nebr['Neighborhood']=Neig

Nebr

NameError: name 'Toronto' is not defined

# Data Cleaning
If Borough is Not Assigned drop row.

Reset Index

In [6]:
Nebr = Nebr.drop(Nebr[Nebr['Borough'].str.contains("Not assigned")==True].index, axis=0, inplace=False)

Nebr.index = pd.RangeIndex(len(Nebr.index))
Nebr

,Postal_Code,Borough,Neighborhood


In [7]:
Nebr.shape

(0, 3)

In [8]:
column_n = ['NEBRVAL'] 
NEBR_NA = pd.DataFrame(columns=column_n)
NA=['Not assigned'] 
NEBR_NA['NEBRVAL'] = NA

Nebr1=Nebr

for row_index,row in Nebr.iterrows():
    if((Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned') or (Nebr.loc[row_index,['Neighborhood']].values.astype('str') == 'Not assigned\n')):
       
        Nebr1.loc[row_index,['Neighborhood']] = Nebr1.loc[row_index,['Borough']].values.astype('str') 
        
#Reset Index
#nbr.index = pd.RangeIndex(len(nbr.index))

Nebr1

,Postal_Code,Borough,Neighborhood


In [9]:
Nebr2=Nebr1.groupby('Postal_Code').agg({'Borough':'first',
                               'Neighborhood': ', '.join}).reset_index()

column_names = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr3 = pd.DataFrame(columns=column_names)

Nebr3 = Nebr2.drop(Nebr2[Nebr2['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)

#Reset Index
Nebr3.index = pd.RangeIndex(len(Nebr3.index))
Nebr3

,Postal_Code,Borough,Neighborhood


In [10]:
column = ['Postal_Code','Borough', 'Neighborhood'] 
Nebr_ungrp = pd.DataFrame(columns=column_names)

Nebr_ungrp = Nebr1.drop(Nebr1[Nebr1['Borough'].str.contains("Toronto")==False].index, axis=0, inplace=False)


Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))
Nebr_ungrp

,Postal_Code,Borough,Neighborhood


In [11]:
#!conda install -c conda-forge geopy --yes 
import time
from geopy.geocoders import Nominatim

In [12]:
from geopy.util import get_version
get_version()

'2.0.0'

In [13]:
geolocator = Nominatim(scheme='http', user_agent="ES1234")

for row_index, item in Nebr_ungrp.iterrows():
    
    list1 = Nebr_ungrp.loc[[row_index],['Neighborhood']].values.astype('str')
    loc = ' , Toronto, Ontario, Canada'
    list1.astype('str')
    list1 = np.append(list1, loc)
    latitude = None
    longitude = None
    location = None
    
    location = geolocator.geocode(list1 , limit = 15)
    time.sleep(5)
    if(location is not None):
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Latitude'] = location.latitude
        Nebr_ungrp.loc[Nebr_ungrp.index[row_index], 'Longitude'] = location.longitude

In [14]:
print(Nebr_ungrp)

Empty DataFrame
Columns: [Postal_Code, Borough, Neighborhood]
Index: []


In [15]:
import json 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Importing to use the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes  #Uncomment if not installed
import folium

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: - 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
                                                                                                            \failed

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - cffi -> python[version='2.7.*|3.5.*|3.6.*|3.6.9|3.6.9|3.6.9|3.6.9|>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.9,<3.10.0a0|>=3.8,<3.9.0a0|>=2.7,<2.8.0a0|>=3.5,<3.6.0a0|3.4.*',build='3_73_pypy|2_73_pypy|1_73_pypy|0_73_pypy']
  - rsa -> python[version='2.7.*|3.4.*|3.5.*|3.6.*']

Your python: python=3.7

If python is on the left-most side of the chain, that's the version you've asked for.
When python appears to the right, that indic

ModuleNotFoundError: No module named 'folium'

In [17]:
print('We have {} boroughs and {} neighborhoods.'.format(
        len(Nebr_ungrp['Borough'].unique()),
        Nebr_ungrp.shape[0]
    )
)

Nebr_ungrp.dropna(inplace =True)
Nebr_ungrp.index = pd.RangeIndex(len(Nebr_ungrp.index))

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ES1234")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

We have 0 boroughs and 0 neighborhoods.
The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [18]:
Nebr_ungrp

,Postal_Code,Borough,Neighborhood


In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Nebr_ungrp['Latitude'], Nebr_ungrp['Longitude'], Nebr_ungrp['Borough'], Nebr_ungrp['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
# map_toronto

NameError: name 'folium' is not defined

# Lets use FOURSQUARE API to explore the neighbourhood

In [1]:
CLIENT_ID = 'QEF4SFROUUVOQKNPAFNNRDUW4ACAWSYYG312LM3BKEDDKPIZ' # your Foursquare ID
CLIENT_SECRET = 'CCBFW1WDVTUYDLPFX3SR0QBB5R2UKRYOZ3F1JDJ3PLQMPDRM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Successfully Logged-In')

Successfully Logged-In


In [2]:
Nebr_ungrp.loc[0]
neighborhood_latitude = np.float(Nebr_ungrp.loc[0,['Latitude']].values)
neighborhood_longitude =  np.float(Nebr_ungrp.loc[0,['Longitude']].values)

NameError: name 'Nebr_ungrp' is not defined

Now, let's get the top 100 venues that are in Harbour Square Park within a radius of 500 meters.
First, let's create the GET request URL. Name the URL url

In [3]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

NameError: name 'neighborhood_latitude' is not defined

In [4]:
results = requests.get(url).json()
results

NameError: name 'requests' is not defined

In [5]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [ ]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

And how many venues were returned by Foursquare?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

# 2. Explore Neighborhoods in Toronto

In [ ]:

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
toronto_venues = getNearbyVenues(names=Nebr_ungrp['Neighborhood'],
                                   latitudes=Nebr_ungrp['Latitude'],
                                   longitudes=Nebr_ungrp['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

# 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
Top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(Top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, Top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:Top_venues]

In [ ]:
Top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(Top_venues):
    try:
        columns.append('{}{} Popular Venues'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Popular Venues'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], Top_venues)

neighborhoods_venues_sorted

# 4. Cluster Neighborhoodsusing K-Mean

In [ ]:

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood',1)
#print(toronto_grouped_clustering)
#print(toronto_grouped)
# run k-means clustering
kmeans = KMeans(init = "k-means++", n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
labels = kmeans.labels_[0:63] 
print(labels)

In [ ]:
toronto_merged = Nebr_ungrp
print(toronto_merged.shape)
labels = np.append(labels,labels[0])
print(labels.shape)
# add clustering labels
toronto_merged['Cluster Labels'] = labels.tolist()

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 5. Examine Clusters

## Cluster 1

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 2

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 3

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 4

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

## Cluster 5

In [ ]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]